In [1]:
import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
IMAGE_SIZE = 448
NUM_CLASSES = 20
BOXES_PER_CELL = 2
CELL_SIZE = 7

In [4]:
def conv_leaky_relu(inputs, filters, size, stride):
    x = tf.keras.layers.Conv2D(filters, size, stride, padding="same",)(inputs)
    x = tf.keras.layers.LeakyReLU(0.1)(x)
    return x

In [5]:
def residual_block(inputs, filters, size, stride, kernel_init: str='zeros'):
    x = tf.keras.layers.Conv2D(
        filters, size, stride, padding="same",
        kernel_initializer=kernel_init
    )(inputs)
    x = tf.keras.layers.LeakyReLU(0.1)(x)
    x = x + inputs
    return x

In [7]:
def yolo():
    yolo_input = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    pretrained_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    pretrained_model.trainable = False

    x = pretrained_model(yolo_input)

    x = conv_leaky_relu(x, 1024, 3, 1)
    x = tf.keras.layers.BatchNormalization()(x)
    x = conv_leaky_relu(x, 1024, 3, 2)
    x = tf.keras.layers.BatchNormalization()(x)

    x = conv_leaky_relu(x, 1024, 3, 1)
    x = tf.keras.layers.BatchNormalization()(x)
    x = conv_leaky_relu(x, 1024, 3, 1)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(4096, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)
    x = tf.keras.layers.LeakyReLU(0.1)(x)

    yolo_outputs = tf.keras.layers.Dense(CELL_SIZE * CELL_SIZE * (NUM_CLASSES + BOXES_PER_CELL * 5),
                            kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)

    return tf.keras.Model(inputs=yolo_input, outputs=yolo_outputs, name="YOLO")

def new_yolo():
    yolo_input = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    pretrained_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

    pretrained_model.trainable = False

    x = pretrained_model(yolo_input)

    x = conv_leaky_relu(x, 1024, 3, 1)
    x = tf.keras.layers.BatchNormalization()(x)
    x = conv_leaky_relu(x, 1024, 3, 2)
    x = tf.keras.layers.BatchNormalization()(x)

    x = conv_leaky_relu(x, 1024, 3, 1)
    x = tf.keras.layers.BatchNormalization()(x)
    x = conv_leaky_relu(x, 1024, 3, 1)
    x = tf.keras.layers.BatchNormalization()(x)

    # Add 2 layers without changing the model
    x = residual_block(x, 1024, 3, 1)
    x = tf.keras.layers.BatchNormalization()(x)
    x = residual_block(x, 1024, 3, 1)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(4096, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)
    x = tf.keras.layers.LeakyReLU(0.1)(x)

    yolo_outputs = tf.keras.layers.Dense(CELL_SIZE * CELL_SIZE * (NUM_CLASSES + BOXES_PER_CELL * 5),
                            kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)

    return tf.keras.Model(inputs=yolo_input, outputs=yolo_outputs, name="new-YOLO")

In [8]:
# reload_from_ckpt = '../ckpts/YOLO-Resnet50-from-ep8-ep32/yolo-31'
reload_from_ckpt = './ckpt-load/yolo-20'


old_YOLO = yolo()
# new_YOLO = new_yolo()

if reload_from_ckpt is not None:
    ckpt = tf.train.Checkpoint(net=old_YOLO)
    ckpt.restore(reload_from_ckpt)

In [9]:
for var in tf.train.list_variables(reload_from_ckpt):
    print(var)  # 檢查權重的名稱和形狀

('_CHECKPOINTABLE_OBJECT_GRAPH', [])
('epoch/.ATTRIBUTES/VARIABLE_VALUE', [])
('net/layer_with_weights-0/layer_with_weights-0/bias/.ATTRIBUTES/VARIABLE_VALUE', [64])
('net/layer_with_weights-0/layer_with_weights-0/kernel/.ATTRIBUTES/VARIABLE_VALUE', [7, 7, 3, 64])
('net/layer_with_weights-0/layer_with_weights-1/beta/.ATTRIBUTES/VARIABLE_VALUE', [64])
('net/layer_with_weights-0/layer_with_weights-1/gamma/.ATTRIBUTES/VARIABLE_VALUE', [64])
('net/layer_with_weights-0/layer_with_weights-1/moving_mean/.ATTRIBUTES/VARIABLE_VALUE', [64])
('net/layer_with_weights-0/layer_with_weights-1/moving_variance/.ATTRIBUTES/VARIABLE_VALUE', [64])
('net/layer_with_weights-0/layer_with_weights-10/bias/.ATTRIBUTES/VARIABLE_VALUE', [64])
('net/layer_with_weights-0/layer_with_weights-10/kernel/.ATTRIBUTES/VARIABLE_VALUE', [1, 1, 256, 64])
('net/layer_with_weights-0/layer_with_weights-100/bias/.ATTRIBUTES/VARIABLE_VALUE', [512])
('net/layer_with_weights-0/layer_with_weights-100/kernel/.ATTRIBUTES/VARIABLE_VALU

In [10]:
for idx, layer in enumerate(old_YOLO.layers):
    print(idx, layer)

0 <keras.engine.input_layer.InputLayer object at 0x0000027327CDEBB0>
1 <keras.engine.functional.Functional object at 0x000002732F9763D0>
2 <keras.layers.convolutional.conv2d.Conv2D object at 0x000002732F9658B0>
3 <keras.layers.activation.leaky_relu.LeakyReLU object at 0x000002732F956A30>
4 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000002732F956C10>
5 <keras.layers.convolutional.conv2d.Conv2D object at 0x000002732F94CFA0>
6 <keras.layers.activation.leaky_relu.LeakyReLU object at 0x000002732F96CDF0>
7 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000002733FE60160>
8 <keras.layers.convolutional.conv2d.Conv2D object at 0x000002732F94C5E0>
9 <keras.layers.activation.leaky_relu.LeakyReLU object at 0x000002732F9569D0>
10 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000002733FE70AC0>
11 <keras.layers.convolutional.conv2d.Conv2D object at 0x000002733FE60CD0>
12 <keras.layers.activation.leaky_

In [ ]:
old_YOLO_indices = [i for i, layer in enumerate(old_YOLO.layers)]
for _ in range(6):
    old_YOLO_indices.insert(14, -1)

new_YOLO_indices = [i for i, layer in enumerate(new_YOLO.layers)]

for old_id, new_id in zip(old_YOLO_indices, new_YOLO_indices):
    if old_id == -1:
        continue

    old_layer = old_YOLO.layers[old_id]
    new_layer = new_YOLO.layers[new_id]

    print("{}: Setting layer {} to layer: {}".format(new_id, old_layer.name, new_layer.name))

    new_layer.set_weights(old_layer.get_weights())

0: Setting layer input_1 to layer: input_3
1: Setting layer resnet50 to layer: resnet50
2: Setting layer conv2d to layer: conv2d_4
3: Setting layer leaky_re_lu to layer: leaky_re_lu_5
4: Setting layer batch_normalization to layer: batch_normalization_4
5: Setting layer conv2d_1 to layer: conv2d_5
6: Setting layer leaky_re_lu_1 to layer: leaky_re_lu_6
7: Setting layer batch_normalization_1 to layer: batch_normalization_5
8: Setting layer conv2d_2 to layer: conv2d_6
9: Setting layer leaky_re_lu_2 to layer: leaky_re_lu_7
10: Setting layer batch_normalization_2 to layer: batch_normalization_6
11: Setting layer conv2d_3 to layer: conv2d_7
12: Setting layer leaky_re_lu_3 to layer: leaky_re_lu_8
13: Setting layer batch_normalization_3 to layer: batch_normalization_7
20: Setting layer flatten to layer: tf.__operators__.add_1
21: Setting layer dense to layer: batch_normalization_9


ValueError: You called `set_weights(weights)` on layer "batch_normalization_9" with a weight list of length 2, but the layer was expecting 4 weights. Provided weights: [array([[ 0.0073933 , -0.00091103, -0.00607041, .....

In [ ]:
ckpt = tf.train.Checkpoint(epoch=tf.Variable(0), net=new_YOLO)

# manager = tf.train.CheckpointManager(ckpt, './ckpts/new-YOLO-Resnet50', max_to_keep=3,
#                                      checkpoint_name='yolo')
manager = tf.train.CheckpointManager(ckpt, './ckpt-save', max_to_keep=3,
                                     checkpoint_name='yolo')

save_path = manager.save()